In [1]:
%reset -f
%load_ext autoreload
%autoreload 2
# import sys
# sys.path.append('/home/saptarshi/Dhruv/Dissertation/ICC/')
from generate_attributions import *
from utils import CONFIG,evaluate_exp,PREFIX,aggregate_auc_or_sum

2025-06-04 16:20:22.557565: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749034222.582263   47340 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749034222.589624   47340 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-04 16:20:22.613022: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/dhruv/miniconda3/envs/Dhruv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please upda

## German Credit Dataset

In [3]:
config = CONFIG['german']
mlp_path = PREFIX + 'models/german_mlp/german_mlp_seed_1/logs/version_2/checkpoints/epoch=8-step=108.ckpt'
scm_path = PREFIX + 'models/german_nf/german_nf_seed_2/logs/version_1/checkpoints/epoch=131-step=1584.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_mlp'],classification=config['classification']))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))

In [3]:
##generate attributions
ge_dict, time_dict, global_explanations = generate_global_exps(config,mlp_model.to('cpu'),scm_model.to('cpu'))

Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1


In [4]:
# Load JSON
with open(PREFIX + "attributions/german/attribution.json", "r") as f:
    data = json.load(f)
global_explanations = data['global_explanations']    
generate_attr_plot(global_explanations,config)

In [28]:
ge_dict = data["ge_dict"]
evaluation_metrics,evaluation_metrics_per_sample = evaluate_exp(ge_dict,config,mlp_model.to('cpu'))
print(evaluation_metrics)


{'shap': {'pgi': {'k=1': [0.19377297163009644], 'k=2': [0.191986083984375], 'k=3': [0.204826220870018], 'k=4': [0.21538521349430084]}, 'pgu': {'k=1': [0.15497887134552002], 'k=2': [0.1511467546224594], 'k=3': [0.1238788589835167], 'k=4': [1.3307929869199597e-08]}}, 'lime': {'pgi': {'k=1': [0.1940106451511383], 'k=2': [0.20758767426013947], 'k=3': [0.205860435962677], 'k=4': [0.2156440168619156]}, 'pgu': {'k=1': [0.1550999879837036], 'k=2': [0.11121926456689835], 'k=3': [0.10055713355541229], 'k=4': [1.3307929869199597e-08]}}, 'sp_lime': {'pgi': {'k=1': [0.049449991434812546], 'k=2': [0.12903520464897156], 'k=3': [0.15521664917469025], 'k=4': [0.21537992358207703]}, 'pgu': {'k=1': [0.21671603620052338], 'k=2': [0.20543433725833893], 'k=3': [0.1944667398929596], 'k=4': [1.3307929869199597e-08]}}, 'pfi': {'pgi': {'k=1': [0.1940975934267044], 'k=2': [0.20797312259674072], 'k=3': [0.20598606765270233], 'k=4': [0.2155877947807312]}, 'pgu': {'k=1': [0.15517890453338623], 'k=2': [0.11094649136

In [29]:
auc_results,auc_show,sum_results,sum_show = aggregate_auc_or_sum(evaluation_metrics_per_sample,config)

# Print results nicely
for method, vals in auc_show.items():
    print(f"{method}: PGI AUC = {vals['pgi_auc']}, PGU AUC = {vals['pgu_auc']}")


shap: PGI AUC = 0.20046380105945802 +- 0.0014946893134759782, PGU AUC = 0.1175050207393037 +- 0.0012016390367898429
lime: PGI AUC = 0.20609181016683578 +- 0.0016623317274554444, PGU AUC = 0.09644213499294388 +- 0.0007453601416849401
sp_lime: PGI AUC = 0.13888893312050235 +- 0.00139996385936127, PGU AUC = 0.16941970323522887 +- 0.00122483631336689
pfi: PGI AUC = 0.20626728759871588 +- 0.0016449075987225175, PGU AUC = 0.09619710620906617 +- 0.0007484782864640922
ig: PGI AUC = 0.21039588855372535 +- 0.001549906175506227, PGU AUC = 0.07942513097698489 +- 0.0011015520560037077
sg: PGI AUC = 0.21036528567473092 +- 0.0015422088108626164, PGU AUC = 0.07931528958647202 +- 0.0011117082352440674
itg: PGI AUC = 0.2007697810894913 +- 0.0015088732147375784, PGU AUC = 0.11755231110586059 +- 0.0011980093220093767
icc_topo: PGI AUC = 0.20646065026521682 +- 0.0016629901991614545, PGU AUC = 0.09635419192413489 +- 0.0007508354370249216
icc_shap: PGI AUC = 0.2065774295727412 +- 0.0016532201476460928, PGU A

In [30]:
for method, vals in sum_show.items():
    print(f"{method}: PGI SUM = {vals['pgi_sum']}, PGU SUM = {vals['pgu_sum']}")

shap: PGI SUM = 0.8059705495834351 +- 0.006031262188525371, PGU SUM = 0.4300045371055603 +- 0.004180700552818379
lime: PGI SUM = 0.8231027722358704 +- 0.0065471821520538456, PGU SUM = 0.36687642335891724 +- 0.0028113200799610217
sp_lime: PGI SUM = 0.5490818023681641 +- 0.0056254038365409435, PGU SUM = 0.6166172027587891 +- 0.004317368102897732
pfi: PGI SUM = 0.8236444592475891 +- 0.006509055216457779, PGU SUM = 0.36618074774742126 +- 0.0028109605525822477
ig: PGI SUM = 0.8367541432380676 +- 0.0061937837721651335, PGU SUM = 0.31582966446876526 +- 0.003848420604324143
sg: PGI SUM = 0.8363808989524841 +- 0.00619471817834753, PGU SUM = 0.3154735267162323 +- 0.0038803491905761707
itg: PGI SUM = 0.8078094720840454 +- 0.006053632578203294, PGU SUM = 0.43035030364990234 +- 0.0041462713958549295
icc_topo: PGI SUM = 0.8243813514709473 +- 0.00656872824461123, PGU SUM = 0.36626356840133667 +- 0.002808442948424793
icc_shap: PGI SUM = 0.8246815800666809 +- 0.006535556216798773, PGU SUM = 0.366876631

## Lung Cancer Dataset

In [2]:
config = CONFIG['cancer']
mlp_path = PREFIX + 'models/cancer_mlp/cancer_mlp_seed_1/logs/version_0/checkpoints/epoch=33-step=4012.ckpt'
scm_path = PREFIX + 'models/cancer_nf/cancer_nf_seed_3/logs/version_0/checkpoints/epoch=44-step=5310.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_mlp'],classification=config['classification']))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))

In [3]:
##generate attributions
ge_dict, time_dict, global_explanations = generate_global_exps(config,mlp_model.to('cpu'),scm_model.to('cpu'))

Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1


In [4]:
# Load JSON
with open(PREFIX +"attributions/cancer/attribution.json", "r") as f:
    data = json.load(f)
global_explanations = data['global_explanations']    
generate_attr_plot(global_explanations,config)

In [9]:
ge_dict = data["ge_dict"]
evaluation_metrics,evaluation_metrics_per_sample = evaluate_exp(ge_dict,config,mlp_model.to('cpu'))
print(evaluation_metrics)


{'shap': {'pgi': {'k=1': [0.070198193192482], 'k=2': [0.12607547640800476], 'k=3': [0.21975362300872803], 'k=4': [0.23079130053520203], 'k=5': [0.23920278251171112], 'k=6': [0.22352668642997742], 'k=7': [0.22255390882492065]}, 'pgu': {'k=1': [0.21272405982017517], 'k=2': [0.19701719284057617], 'k=3': [0.1530276983976364], 'k=4': [0.14382173120975494], 'k=5': [0.0968848466873169], 'k=6': [0.06978976726531982], 'k=7': [1.6411751957434717e-08]}}, 'lime': {'pgi': {'k=1': [0.22436051070690155], 'k=2': [0.2178170382976532], 'k=3': [0.21964162588119507], 'k=4': [0.22735388576984406], 'k=5': [0.21593312919139862], 'k=6': [0.2256031632423401], 'k=7': [0.2226499617099762]}, 'pgu': {'k=1': [0.17544472217559814], 'k=2': [0.14601799845695496], 'k=3': [0.15316511690616608], 'k=4': [0.12659652531147003], 'k=5': [0.1180243045091629], 'k=6': [0.09519334882497787], 'k=7': [1.6411751957434717e-08]}}, 'sp_lime': {'pgi': {'k=1': [0.06987893581390381], 'k=2': [0.09699656069278717], 'k=3': [0.143850371241569

In [10]:
auc_results,auc_show,sum_results,sum_show = aggregate_auc_or_sum(evaluation_metrics_per_sample,config)

# Print results nicely
for method, vals in auc_show.items():
    print(f"{method}: PGI AUC = {vals['pgi_auc']}, PGU AUC = {vals['pgu_auc']}")


shap: PGI AUC = 0.1976209842059244 +- 0.004060267116491173, PGU AUC = 0.12781720987094258 +- 0.0014003908792349516
lime: PGI AUC = 0.22164234770617136 +- 0.004653718280820448, PGU AUC = 0.12111994352235667 +- 0.0016748436014430023
sp_lime: PGI AUC = 0.15779331975295724 +- 0.0019201276007474997, PGU AUC = 0.16532280416050021 +- 0.003161506619859883
pfi: PGI AUC = 0.22579142216644763 +- 0.004761264491062567, PGU AUC = 0.11622776883535295 +- 0.001710300828269025
ig: PGI AUC = 0.14711400355776358 +- 0.0024766694813069716, PGU AUC = 0.1892505653802202 +- 0.003345981856918145
sg: PGI AUC = 0.15436862682789151 +- 0.002374074571045448, PGU AUC = 0.19764795806802188 +- 0.0031356562622436844
itg: PGI AUC = 0.16100863946275787 +- 0.002211267358643582, PGU AUC = 0.19924058065912686 +- 0.003382810800455524
icc_topo: PGI AUC = 0.22484865179606198 +- 0.004673798588946252, PGU AUC = 0.11383153706457881 +- 0.0016343585941565658
icc_shap: PGI AUC = 0.22546216603792793 +- 0.004765279730632485, PGU AUC = 

In [11]:
for method, vals in sum_show.items():
    print(f"{method}: PGI SUM = {vals['pgi_sum']}, PGU SUM = {vals['pgu_sum']}")

shap: PGI SUM = 1.3321019411087036 +- 0.027208558511949088, PGU SUM = 0.8732653260231018 +- 0.009235138434160323
lime: PGI SUM = 1.5533593893051147 +- 0.03202419744696573, PGU SUM = 0.814441978931427 +- 0.010966321464122655
sp_lime: PGI SUM = 1.093016505241394 +- 0.01373109626251386, PGU SUM = 1.1037219762802124 +- 0.021355429558712127
pfi: PGI SUM = 1.57814359664917 +- 0.032662054532658295, PGU SUM = 0.7850844860076904 +- 0.011179365509130087
ig: PGI SUM = 1.0291413068771362 +- 0.017307060984146146, PGU SUM = 1.2419929504394531 +- 0.022136984780814644
sg: PGI SUM = 1.0706360340118408 +- 0.01635486307081783, PGU SUM = 1.285398006439209 +- 0.02042610866202407
itg: PGI SUM = 1.1163184642791748 +- 0.01525595060987103, PGU SUM = 1.3048359155654907 +- 0.022273765178253996
icc_topo: PGI SUM = 1.5725268125534058 +- 0.032149529332680435, PGU SUM = 0.7706788182258606 +- 0.010821173603550236
icc_shap: PGI SUM = 1.5762704610824585 +- 0.03269055038466711, PGU SUM = 0.7932151556015015 +- 0.01084881

## Synthetic Data

In [5]:
config = CONFIG['syn']
mlp_path = PREFIX + 'models/syn_mlp/syn_mlp_seed_1/logs/version_4/checkpoints/epoch=75-step=8968.ckpt'
scm_path = PREFIX + 'models/syn_nf/syn_nf_seed_1/logs/version_5/checkpoints/epoch=50-step=6018.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_mlp'],classification=config['classification']))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))

In [3]:
##generate attributions
ge_dict, time_dict, global_explanations = generate_global_exps(config,mlp_model.to('cpu'),scm_model.to('cpu'))

Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1


In [6]:
# Load JSON
with open(PREFIX +"attributions/syn/attribution.json", "r") as f:
    data = json.load(f)
global_explanations = data['global_explanations']    
generate_attr_plot(global_explanations,config)

In [14]:
ge_dict = data["ge_dict"]
evaluation_metrics,evaluation_metrics_per_sample = evaluate_exp(ge_dict,config,mlp_model.to('cpu'))
print(evaluation_metrics)


{'shap': {'pgi': {'k=1': [0.005703211296349764], 'k=2': [0.010119044221937656], 'k=3': [0.01724904775619507]}, 'pgu': {'k=1': [0.021173346787691116], 'k=2': [0.01692674681544304], 'k=3': [1.410643246657628e-08]}}, 'lime': {'pgi': {'k=1': [0.01695871539413929], 'k=2': [0.012024368159472942], 'k=3': [0.01725354790687561]}, 'pgu': {'k=1': [0.010111484676599503], 'k=2': [0.0057547446340322495], 'k=3': [1.410643246657628e-08]}}, 'sp_lime': {'pgi': {'k=1': [0.005708939395844936], 'k=2': [0.012017950415611267], 'k=3': [0.017242563888430595]}, 'pgu': {'k=1': [0.0211559496819973], 'k=2': [0.005749939009547234], 'k=3': [1.410643246657628e-08]}}, 'pfi': {'pgi': {'k=1': [0.01693537086248398], 'k=2': [0.012019080109894276], 'k=3': [0.01722407341003418]}, 'pgu': {'k=1': [0.010127478279173374], 'k=2': [0.005754291545599699], 'k=3': [1.410643246657628e-08]}}, 'ig': {'pgi': {'k=1': [0.005744026042521], 'k=2': [0.01012091152369976], 'k=3': [0.017239565029740334]}, 'pgu': {'k=1': [0.012021321803331375], 

In [15]:
auc_results,auc_show,sum_results,sum_show = aggregate_auc_or_sum(evaluation_metrics_per_sample,config)

# Print results nicely
for method, vals in auc_show.items():
    print(f"{method}: PGI AUC = {vals['pgi_auc']}, PGU AUC = {vals['pgu_auc']}")


shap: PGI AUC = 0.010797586934718615 +- 5.524551590331293e-05, PGU AUC = 0.013756713514138635 +- 7.532188219725576e-05
lime: PGI AUC = 0.014565249162726104 +- 7.497788546115038e-05, PGU AUC = 0.0054052474921530425 +- 5.780337398376364e-05
sp_lime: PGI AUC = 0.011746850913390517 +- 5.680071442689552e-05, PGU AUC = 0.008163960050374347 +- 4.080233701449235e-05
pfi: PGI AUC = 0.014549400687062492 +- 7.490479049653797e-05, PGU AUC = 0.005409018908734045 +- 5.77437908243514e-05
ig: PGI AUC = 0.01080635379976593 +- 6.417266438571556e-05, PGU AUC = 0.011481282443817084 +- 6.803728349199149e-05
sg: PGI AUC = 0.010806780933324869 +- 6.415090022513764e-05, PGU AUC = 0.011474482483502167 +- 6.769279835975008e-05
itg: PGI AUC = 0.014564381763804705 +- 7.519143074369308e-05, PGU AUC = 0.005400311663785639 +- 5.754574752635861e-05
icc_topo: PGI AUC = 0.01914022152420754 +- 0.00010059335414699216, PGU AUC = 0.005382174381481794 +- 4.0901126963190483e-05
icc_shap: PGI AUC = 0.019135238598100842 +- 0.0

In [16]:
for method, vals in sum_show.items():
    print(f"{method}: PGI SUM = {vals['pgi_sum']}, PGU SUM0= {vals['pgu_sum']}")

shap: PGI SUM = 0.033071305602788925 +- 0.0001322619125211425, PGU SUM0= 0.03810010850429535 +- 0.00020850252340285727
lime: PGI SUM = 0.04623662680387497 +- 0.0002232661537661454, PGU SUM0= 0.015866246074438095 +- 0.0001659949315415093
sp_lime: PGI SUM = 0.034969452768564224 +- 0.00015307530616324227, PGU SUM0= 0.026905899867415428 +- 0.0001332920347433755
pfi: PGI SUM = 0.04617852345108986 +- 0.00022296484929832836, PGU SUM0= 0.01588178426027298 +- 0.0001660684060667619
ig: PGI SUM = 0.03310450166463852 +- 0.00016390749034100974, PGU SUM0= 0.028973231092095375 +- 0.0001706579367990302
sg: PGI SUM = 0.03310207277536392 +- 0.00016368010526138663, PGU SUM0= 0.02896091900765896 +- 0.0001699633374211269
itg: PGI SUM = 0.046230584383010864 +- 0.00022357284468254906, PGU SUM0= 0.01585792377591133 +- 0.00016549182762485586
icc_topo: PGI SUM = 0.05538759380578995 +- 0.0002721420668422512, PGU SUM0= 0.01582576334476471 +- 0.00013078951237198818
icc_shap: PGI SUM = 0.05537247657775879 +- 0.0002

In [ ]:
##Misc
#WD dimensionwise

#Syn
P = [0.0669,0.0640,0.0804]
Q = [0.0415,0.0349,0.0337]
R = [0.0106,0.0064,0.0109] 

#German
gender = [0.0870,0.0916,0.0827]
age = [0.0377,0.0513,0.0406]
amount = [0.0407,0.0530,0.0421]
duration = [0.1074,0.1836,0.1196]

#Cancer
asia = [0.0396,0.0402,0.0427]
tub = [0.0429,0.0456,0.0328]
smoke = [0.0329,0.0366,0.0317]
lung = [0.0317,0.0372,0.0352]
bronc = [0.0432,0.0567,0.0378]
either = [0.0292,0.0275,0.0353]
xray  = [0.0292,0.0654,0.0697]


## Test

In [2]:
config = CONFIG['syn']
mlp_path = '/home/dhruv/Files/Thesis/Dissertation/Code/models/syn_ann/syn_ann_seed_10/logs/version_0/checkpoints/epoch=161-step=5184.ckpt'
scm_path = '/home/dhruv/Files/Thesis/Dissertation/Code/models/syn_nf/syn_nf_seed_10/logs/version_0/checkpoints/epoch=19-step=640.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_ann'],classification=False))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))
ge_dict, evaluation_metrics, time_dict, global_explanations = generate_global_exps(config,mlp_model,scm_model)

Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1


In [3]:
ge_dict

{'ig': {'W': [0.008685602410414437],
  'Z': [0.5541948194138167],
  'X': [0.4371195781757688]},
 'itg': {'W': [0.008485306748260027],
  'Z': [0.497316907775009],
  'X': [0.49419778547673104]},
 'sg': {'W': [0.13136424456458626],
  'Z': [0.3860659244300918],
  'X': [0.48256983100532197]},
 'shap': {'W': [0.5803365821763931],
  'Z': [0.3543153621348831],
  'X': [0.06534805568872387]},
 'lime': {'W': [0.2837432524861816],
  'Z': [0.293367905107146],
  'X': [0.4228888424066725]},
 'sp_lime': {'X': [0.41602805243460733],
  'W': [0.2947343644233361],
  'Z': [0.2892375831420565]},
 'pfi': {'W': [0.2555289566516876],
  'Z': [0.2728649377822876],
  'X': [0.47160604596138]},
 'icc_topo': {'W': [0.07992755621671677],
  'Z': [0.6083893179893494],
  'X': [0.31013962626457214]},
 'icc_shap': {'W': [0.1287594437599182],
  'Z': [0.6630550622940063],
  'X': [0.20664207637310028]}}

In [9]:
generate_attr_plot(global_explanations,config)

In [7]:
# Load JSON
with open("/home/dhruv/Files/Thesis/Dissertation/Code/attributions/syn/attribution.json", "r") as f:
    data = json.load(f)

# Call the function
summary = compute_pgu_pgi_sums(data["evaluation_metrics"])

# Display the results
for method, scores in summary.items():
    print(f"{method}: PGU Sum = {scores['pgu_sum']:.6f}, PGI Sum = {scores['pgi_sum']:.6f}")


ig: PGU Sum = 0.128419, PGI Sum = 0.000436
itg: PGU Sum = 0.128419, PGI Sum = 0.000436
sg: PGU Sum = 0.128418, PGI Sum = 0.000437
shap: PGU Sum = 0.128420, PGI Sum = 0.000435
lime: PGU Sum = 0.128418, PGI Sum = 0.000437
sp_lime: PGU Sum = 0.128420, PGI Sum = 0.000435
pfi: PGU Sum = 0.128418, PGI Sum = 0.000437
icc_topo: PGU Sum = 0.128419, PGI Sum = 0.000436
icc_shap: PGU Sum = 0.128419, PGI Sum = 0.000436
